# Representing pose in 2D
## Daniel Stangeland

In [2]:
# %load robotteknikk.py
#!/usr/bin/env python3
"""
Created on Sun Aug 26 20:07:07 2018

@author: alexalcocer
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
import matplotlib as mpl



def trplot2(T,name='',c='b'):  
    t = T[0:2,2] # translation
    R = T[:2,:2] # rotation  
    X = t + R[:,0] # 
    Y = t + R[:,1]  
    dtext = 0.1
    plt.plot([t[0],X[0]],[t[1],X[1]],color=c,linewidth=2.0)
    plt.plot([t[0],Y[0]],[t[1],Y[1]],color=c,linewidth=2.0)
    plt.text(t[0]-dtext,t[1]-dtext,"{"+name+"}",fontsize=14)
    plt.text(Y[0]+dtext/2,Y[1],r'$Y_{}$'.format(name),fontsize=14)
    plt.text(X[0]+dtext/2,X[1],r'$X_{}$'.format(name),fontsize=14)
    return None
 
def se2(x,y,theta):  
    #theta = np.random.rand()
    R = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    t = np.array([[x],[y]])
    #q = np.concatenate([R,t], axis=1)
    #T = np.concatenate([q,[[0,0,1]]])
    T = np.block([[R,t],[0,0,1]])
    return T

def skew(w):
    """returns a skew symmetric matric from a vector
    w is 3x1 array"""
    w1 = w[0]
    w2 = w[1]
    w3 = w[2]
    return np.array([[0,-w3,w2],[w3,0,-w1],[-w2,w1,0]])   


def trplot3(ax,T,name=None):
    """    T is 3D homogeneous transformation matrix T = [R p; 0 1]
    plots 3 vectors reference frame centered in p and orientation R
    p is a 3 array - origin of frame
    R is a 3x3 array - rotation matrix
    assumes that a figure is currently open with axes ax
    """
    R = T[0:3,0:3] # roation matrix
    p = T[0:3,3]
    X = R + p[:,np.newaxis] # X contains beacon coordinates in "inertial" frame
    dtext = 0.1
    ax.scatter(X[0,:],X[1,:],X[2,:],zdir='z', s=20) # plots beacon positions
    plt.plot([p[0], X[0,0]],[p[1],  X[1,0]],[p[2], X[2,0]],'r',linewidth=2)
    plt.plot([p[0], X[0,1]],[p[1],  X[1,1]],[p[2], X[2,1]],'g',linewidth=2)
    plt.plot([p[0], X[0,2]],[p[1],  X[1,2]],[p[2], X[2,2]],'b',linewidth=2)
    #ax.text(X[0]+dtext/2,X[1],r'$X_{}$'.format(name),fontsize=14)
    if name is not None:
        ax.text(p[0]-dtext,p[1]-dtext,p[2]-dtext, "{"+name+"}",fontsize=12)
        ax.text(X[0,0]+dtext/2,X[1,0],X[2,0], "$X_{}$".format(name),fontsize=10)
        ax.text(X[0,1]+dtext/2,X[1,1],X[2,1], "$Y_{}$".format(name),fontsize=10)
        ax.text(X[0,2]+dtext/2,X[1,2],X[2,2], "$Z_{}$".format(name),fontsize=10)

def e2h(p):
    "Euclidean to homogeneous coordinates"
    # adds a 1, works for 3D and 2D arrays
    return np.append(p,1)

def h2e(ph):
    "Homogeneous to Euclidean coordinates"
    # removes last coordinate, works for 3D and 2D arrays
    return ph[:-1]


def rotx(theta):
    # 3D rotation matrix along x axis
    return np.array([[1,0,0],
                     [0,np.cos(theta),-np.sin(theta)],
                     [0,np.sin(theta),np.cos(theta)]])

def roty(theta):
    # 3D rotation matrix along y axis
    return np.array([[np.cos(theta),0,np.sin(theta)],
                     [0,1,0],
                     [-np.sin(theta),0, np.cos(theta)]])    
    
def rotz(theta):
    # 3D rotation matrix along z axis
    return np.array([[np.cos(theta),-np.sin(theta),0],
                     [np.sin(theta),np.cos(theta),0],
                     [0,0,1]])


def trotx(theta):
    # 3D homogeneous transform matrix rotation matrix along x axis
    R = rotx(theta)
    T = np.eye(4)
    T[0:3,0:3] = R
    return T

def troty(theta):
    # 3D homogeneous transform matrix rotation matrix along x axis
    R = roty(theta)
    T = np.eye(4)
    T[0:3,0:3] = R
    return T

def trotz(theta):
    # 3D homogeneous transform matrix rotation matrix along x axis
    R = rotz(theta)
    T = np.eye(4)
    T[0:3,0:3] = R
    return T

def ttrans(p):
    # 3D homogeneous transform matrix pure translation
    T = np.eye(4)
    T[0:3,3] = p
    return T


def plot_ship(x, theta):
    # Vertex coordinates
    L1 = 3
    L2 = 6
    L3 = 7
    L = L1+L2+L3
    N = 0.2
    W1 = 2.3
    W2 = 2.5
    T = 2.3
    vert = np.array([[0, 0],[-N, 0], [-W1 ,-L1], [-W2 ,-L1-L2],[-T,-L1-L2-L3]])
    vert = vert + np.array([0,L])
    n = vert.shape[0]
    patch = np.zeros((2*n,2))
    patch[:n,:] = vert
    for i in range(n):
        patch[n+i,:] = np.array([-vert[n-i-1,0],vert[n-i-1,1]])
    polygon = patches.Polygon(patch, color="0.5", alpha=1) 
    r = mpl.transforms.Affine2D().rotate(-theta)
    t = mpl.transforms.Affine2D().translate(x[0],x[1])
    tra = r + t + ax.transData
    polygon.set_transform(tra)
    ax.add_patch(polygon)
    


In [3]:
import numpy as np
import matplotlib.pyplot as plt

### Exercise 1
#### a) Using the diagram, determine the positions of the points 1 and 2, denoted by red circles
#### in the following reference frames

##### a. Determine vectors \$^Op_1, ^Ap_1, ^Bp_1, ^Cp_1\$

##### b. Determine vectors \$^Op_2, ^Ap_2, ^Bp_2, ^Cp_2\$


In [4]:
#a.

oP1 = np.array([1,3])
aP1_graph = np.array([0,2])
bB1_graph = np.array([0,-np.sqrt(2)])
cP1_graph = np.array([3,-2])

In [5]:
#b.

oP2 = np.array([4,2])
aP2_graph = np.array([3,1])
bP2_graph = np.array([2*np.sqrt(2),0])
cP2_graph = np.array([0,-1])




#### b) Determine the homogeneous transformation matrices \$^OT_A, ^OT_B, ^OT_C\$

In [6]:
# Find oTa
oTa = se2(1,1,0)
oTa 

array([[ 1., -0.,  1.],
       [ 0.,  1.,  1.],
       [ 0.,  0.,  1.]])

In [7]:
# Find oTb
oTb = se2(2,4,-np.pi/4)
oTb

array([[ 0.70710678,  0.70710678,  2.        ],
       [-0.70710678,  0.70710678,  4.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [8]:
# Find oTc
oTc = se2(4,1,-np.pi)
oTc

array([[-1.0000000e+00,  1.2246468e-16,  4.0000000e+00],
       [-1.2246468e-16, -1.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])




#### c) Determine the homogeneous transformation matrices \$^BT_C, ^CT_A, ^AT_B\$

In [9]:
# Find bTc
bTc = se2((5*np.sqrt(2))/2,-(np.sqrt(2))/2,(np.pi*5)/4)
bTc

array([[-0.70710678,  0.70710678,  3.53553391],
       [-0.70710678, -0.70710678, -0.70710678],
       [ 0.        ,  0.        ,  1.        ]])

In [10]:
# Find cTa
cTa = se2(3,0,np.pi)
cTa

array([[-1.0000000e+00, -1.2246468e-16,  3.0000000e+00],
       [ 1.2246468e-16, -1.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])

In [11]:
# Find aTb
aTb = se2(1,3,-np.pi/4)
aTb

array([[ 0.70710678,  0.70710678,  1.        ],
       [-0.70710678,  0.70710678,  3.        ],
       [ 0.        ,  0.        ,  1.        ]])




#### d) Using the obtained transformation matrices \$^OT_A, ^OT_B, ^OT_C\$ and \$^Op_1, ^Op_2\$ from the
#### diagram, determine the position vectors

a. \$^Ap_1, ^Bp_1, ^Cp_1\$ 

b. \$^Ap_2, ^Bp_2, ^Cp_2\$

In [12]:
# Create a function that returns the desired position vector using
# a known reference vector and the homogeneous transfer matrix

def newVec(aTb,aP1):
    aP1_h = np.append(aP1,1) # coverts known vector to homogeneous, adds 1
    bP1_h = np.linalg.inv(aTb).dot(aP1_h) # find the new vector
    bP1 = bP1_h[:-1]
    return bP1

In [13]:
# a.
# Find aP1
aP1 = newVec(oTa,oP1)
aP1

array([0., 2.])

In [14]:
# find bP1
bP1 = newVec(oTb,oP1)
bP1

array([-2.22044605e-16, -1.41421356e+00])

In [15]:
# Find cP1
cP1 = newVec(oTc,oP1)
cP1

array([ 3., -2.])

In [16]:
# b.
# Find aP2
aP2 = newVec(oTa,oP2)
aP2

array([3., 1.])

In [17]:
# Find bP2
bP2 = newVec(oTb,oP2)
bP2

array([2.82842712, 0.        ])

In [18]:
# Find cP2
cP2 = newVec(oTc,oP2)
cP2

array([ 0., -1.])



### Exercise 2 
#### Determine if the following are valid rotation matrices. 

HINT: A rotation matrix satisfies:
\$det(R) = 1\$ and \$R^TR = I\$

a. \$R = \begin{bmatrix}0.995 & -0.0998\\ 0.0998 & 0.995\end{bmatrix}\$

In [19]:
# Create a funcion that checks if the current matrix satisfies the two statements above. Since we use a computer
# this will be approximate

def isRotMat(R):
    Rdim = R.shape
    RtR = R.T.dot(R)
    tol = 1e-5
    detR = np.linalg.det(R)
    
    #checks if RtR and detR is close to the identity matrix and 1.
    if np.allclose(RtR,np.identity(Rdim[0]), rtol=tol, atol=tol) and np.isclose(detR,1,rtol=tol, atol=tol):
        print("R =\n",R,"Is a valid rotation matrix\n")
        print("det(R) = ",detR,"\n")
        print("R^TR = \n",RtR)
    else:
        print("R =\n", R,"Is NOT avalid rotation matrix\n")
        print("det(R) = ",detR,"\n")
        print("R^TR = \n",RtR)
    return 0

In [20]:
#a
isRotMat(np.array([[0.995,-0.0998],[0.0998, 0.995]]))

R =
 [[ 0.995  -0.0998]
 [ 0.0998  0.995 ]] Is a valid rotation matrix

det(R) =  0.9999850399999999 

R^TR = 
 [[ 9.99985040e-01 -9.98312544e-19]
 [-9.98312544e-19  9.99985040e-01]]


0

In [21]:
# b. and e.
isRotMat(np.array([[1,0],[0,1]]))

R =
 [[1 0]
 [0 1]] Is a valid rotation matrix

det(R) =  1.0 

R^TR = 
 [[1 0]
 [0 1]]


0

In [22]:
# c.
isRotMat(np.array([[1, 0],[0,-1]]))

R =
 [[ 1  0]
 [ 0 -1]] Is NOT avalid rotation matrix

det(R) =  -1.0 

R^TR = 
 [[1 0]
 [0 1]]


0

In [23]:
# d.
isRotMat(np.array([[1,1],[0,2]]))

R =
 [[1 1]
 [0 2]] Is NOT avalid rotation matrix

det(R) =  2.0 

R^TR = 
 [[1 1]
 [1 5]]


0

In [24]:
# f. and g.
a = np.sqrt(2)/2
isRotMat(np.array([[a,-a],[a,a]]))

R =
 [[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]] Is a valid rotation matrix

det(R) =  1.0 

R^TR = 
 [[ 1.00000000e+00 -4.26642159e-17]
 [-4.26642159e-17  1.00000000e+00]]


0

In [25]:
# h. and i.
isRotMat(np.array([[-0.5,-0.866],[0.866,-0.5]]))

R =
 [[-0.5   -0.866]
 [ 0.866 -0.5  ]] Is NOT avalid rotation matrix

det(R) =  0.999956 

R^TR = 
 [[0.999956 0.      ]
 [0.       0.999956]]


0



### Exercise 3 
#### Consider a rotation matrix, \$R(\theta) = \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix} \$

#### a. Show that 𝑅(0) = \$I_2\$ where \$I_2\$ is the identity matrix of dimension 2.

Given:

\$I_2 = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}\$

\$\sin{(0)} = 0\\
\cos{(0)} = 1\$

Solution:

\$R(0) = \begin{bmatrix}\cos{(0)} & -\sin{(0)}\\ \sin{(0)} & \cos{(0)}\end{bmatrix} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} = I_2\$





#### b. Show that \$det(𝑅{(\theta)})=+1\$ for any angle \$\theta\$

Given:

The 2x2 rotation matrix 

\$R(\theta) = \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix} \$

Solution:

We can show that \$det(𝑅{(\theta)})=+1\$ for any angle \$\theta\$ by using the trigonometric identity \$\cos^2(\theta) + \sin^2(\theta) = 1\$ in combination with the equation for finding the determinant of a 2x2 matrix \$\det(A)=\frac{1}{ad-bc}\$ where \$ \begin{bmatrix}a & b \\ c & d\end{bmatrix} = \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix} \$

\$\det(𝑅{(\theta)})=\frac{1}{\cos(\theta)\cdot\cos(\theta)-(-\sin(\theta)\cdot\sin(\theta))} = \frac{1}{\cos^2(\theta)+\sin^2(\theta)} = \frac{1}{1} = 1 \$




#### c. Show that \$R{(\theta)}^TR(\theta) = I_2\$

Given:

\$\cos^2(\theta) + \sin^2(\theta) = 1\$

\$R(\theta) = \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix} \$

\$R(\theta)^T = \begin{bmatrix}\cos{\theta} & \sin{\theta}\\ -\sin{\theta} & \cos{\theta}\end{bmatrix} \$

Solution:

\$R{(\theta)}^TR(\theta) = \begin{bmatrix}\cos{\theta} & \sin{\theta}\\ -\sin{\theta} & \cos{\theta}\end{bmatrix} \cdot \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix} =  \begin{bmatrix}{\cos^2{\theta}+\sin^2{\theta} } & -{\cos{\theta}\sin{\theta}} +{\cos{\theta}\sin{\theta}} \\ -{\cos{\theta}\sin{\theta}} +{\cos{\theta}\sin{\theta}} & {\cos^2{\theta}+\sin^2{\theta} }\end{bmatrix} = \begin{bmatrix} 1 && 0\\ 0 && 1 \end{bmatrix} = I_2\$




#### d. Show that \$R{(-\theta)}=R(\theta)^T\$

Given:

According to opposite angles we can express

\$\cos{-\theta} = \cos{\theta}\\
\sin{-\theta} = -\sin{\theta}\$

Solution:

\$R(-\theta) = \begin{bmatrix}\cos{(-\theta)} & -\sin{(-\theta)}\\ \sin{(-\theta)} & \cos{(-\theta)}\end{bmatrix} = \begin{bmatrix}\cos{\theta} & \sin{\theta}\\ -\sin{\theta} & \cos{\theta}\end{bmatrix} = R(\theta)^T \$




#### e. Show that R(a)R(b) = R(a+b)

Given:

\$R(a) = \begin{bmatrix}\cos{a} & -\sin{a}\\ \sin{a} & \cos{a}\end{bmatrix} \\
R(b) = \begin{bmatrix}\cos{b} & -\sin{b}\\ \sin{b} & \cos{b}\end{bmatrix}  \$

\$
\sin(a\pm b) = \sin a \cdot cos b \pm \cos a \cdot \sin b\\
\cos(a\pm b) = \cos a \cdot cos b \mp \sin a \cdot \sin b
\$

Solution:

\$
R(a)R(b) = \begin{bmatrix}\cos{a} & -\sin{a}\\ \sin{a} & \cos{a}\end{bmatrix} \cdot \begin{bmatrix}\cos{b} & -\sin{b}\\ 
\sin{b} & \cos{b}\end{bmatrix} = \begin{bmatrix} (cos(a) cos(b) - sin(a) sin(b)) & (-cos(a) sin(b) - cos(b) sin(a)) \\
(cos(b) sin(a) + cos(a) sin(b)) & (-sin(b) sin(a) + cos(a) cos(b))\end{bmatrix}=\$
\$
\begin{bmatrix} cos(a+b) & -(cos(a) sin(b) + cos(b) sin(a)) \\
sin(a+b) & cos(a+b)\end{bmatrix} = 
\begin{bmatrix} cos(a+b) & -sin(a+b) \\
sin(a+b) & cos(a+b)\end{bmatrix} = R(a+b)
\$




#### f. Show that the columns of $R(\theta)$ are orthonormal (ortogonal and unit norm)

##### Given:

\$R = \begin{bmatrix} \mathbf{a} & \mathbf{b}\end{bmatrix} = \begin{bmatrix} a_1 & b_1\\ a_2 & b_2\end{bmatrix} = \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix}\$

The columns are unit norm if the lengths \$||\mathbf{a}||^2=\sqrt{a_{1}^2 +a_{2}^2} = 1\$ and \$||\mathbf{b}||^2=\sqrt{b_{1}^2 +b_{2}^2} = 1\$

The columns are ortogonal if \$\mathbf{a^Tb} = 0\$

##### Solution:

Unit norm:

Column \$\mathbf{a}\$:

\$||\mathbf{a}||^2  = \sqrt{a_{1}^2 +a_{2}^2} = \sqrt{cos^2(\theta) +sin^2(\theta)} = \sqrt{1} = 1\$

Column \$\mathbf{b}\$:

\$||\mathbf{b}||^2  = \sqrt{b_{1}^2 +b_{2}^2} = \sqrt{(-sin(\theta))^2 +cos^2(\theta)} = \sqrt{1} = 1\$

Ortogonal:

\$\mathbf{a^Tb} = \begin{bmatrix}cos(\theta) & sin(\theta)\end{bmatrix} \cdot \begin{bmatrix}-sin(\theta) \\ cos(\theta)\end{bmatrix} = -sin(\theta)cos(\theta) + sin(\theta)cos(\theta) = 0\$




#### g. Show that the rows of \$R(\theta)\$ are orthonormal

##### Solution:

Unit norm:

Row \$a_{1}b_{1}\$:

\$||a_{1}b_{1}||^2 = \sqrt{a_{1}^2 +b_{1}^2} = \sqrt{cos^2(\theta) + (-sin(\theta))^2} = \sqrt{1} = 1\$

Row \$a_{2}b_{2}\$:

\$||a_{2}b_{2}||^2 = \sqrt{a_{2}^2 +b_{2}^2} = \sqrt{sin^2(\theta) + cos^2(\theta)} = \sqrt{1} = 1\$

Ortogonal:

\${a_{1}b_{1}}^T \cdot a_{2}b_{2} = \begin{bmatrix}cos(\theta) \\ -sin(\theta)\end{bmatrix} \cdot \begin{bmatrix}sin(\theta) & cos(\theta)\end{bmatrix} = sin(\theta)cos(\theta) - sin(\theta)cos(\theta) = 0\$



### Exercise 4
Consider the transforms

\$T_1 = \begin{bmatrix} R_1 & t_1 \\ 0_{1x2} & 1 \end{bmatrix}\$
\$T_2 = \begin{bmatrix} R_2 & t_2 \\ 0_{1x2} & 1 \end{bmatrix}\$

where

\$t_1 = \begin{bmatrix} x_1 \\ y_1 \end{bmatrix}\$
\$t_2 = \begin{bmatrix} x_2 \\ y_2 \end{bmatrix}\$
\$0_{1x2} = \begin{bmatrix} 0 & 0 \end{bmatrix}\$

show that 

\$T_{1}T_{2} =\begin{bmatrix} R_{1}R_{2} & t_1 + R_{1}t_{2} \\ 0_{1x2} & 1 \end{bmatrix} \$


Since the dimenions for each "inner" matrix is we can multiply

\$T_1 \cdot T_2 = \begin{bmatrix} R_1 & t_1 \\ 0_{1x2} & 1 \end{bmatrix} \cdot \begin{bmatrix} R_2 & t_2 \\ 0_{1x2} & 1 \end{bmatrix} = \begin{bmatrix} R_{1}R_{2} + (t_{1}\cdot0_{1X2}) & t_1+R_{1}t_2 \\ (0_{1X2}\cdot{R_2}) + 0_{1X2} & (0_{1X2}\cdot t_2)+1 \end{bmatrix} \$

Computing the different "inner" matrix multiplications

\$(t_1 \cdot 0_{1X2}) = \begin{bmatrix} x_1 \\ y_1 \end{bmatrix} \begin{bmatrix} 0 & 0 \end{bmatrix} = \begin{bmatrix} 0 & 0 \\ 0 & 0 \end{bmatrix}\$

\$(0_{1X2} \cdot R_2) = \begin{bmatrix} 0 & 0 \end{bmatrix} \begin{bmatrix} cos(\theta) & -sin(\theta) \\ sin(\theta) & cos(\theta)  \end{bmatrix} = \begin{bmatrix} 0 & 0 \end{bmatrix}\$

\$(0_{1X2} \cdot t_2)  = \begin{bmatrix} 0 & 0 \end{bmatrix} \begin{bmatrix} x_2 \\ y_2 \end{bmatrix} = 0\$


This means that we express \$T_{1}T_{2}\$ as 

\$T_1 \cdot T_2 = \begin{bmatrix} R_{1}R_{2} + (t_{1}\cdot0_{1X2}) & t_1+R_{1}t_2 \\ (0_{1X2}\cdot{R_2}) + 0_{1X2} & (0_{1X2}\cdot t_2)+1 \end{bmatrix} =  \begin{bmatrix} R_{1}R_{2} & t_1 + R_{1}t_{2} \\ 0_{1x2} & 1 \end{bmatrix}\$




### Exercise 5

Show that \$T^{-1} = \begin{bmatrix} R^T & -R^T t \\ 0_{1X2} & 1 \end{bmatrix}\$ is the inverse transform of \$T = \begin{bmatrix} R & t \\ 0_{1X2} & 1 \end{bmatrix}\$

By using the definition for the invertibe matrix, \$A\$ is \$A^{-1}\$ only when \$A \cdot A^{-1} = A^{-1} \cdot A = I_n\$, we want to show that \$T^{-1}\$ is the invertible matrix of \$T\$.

\$\\ \$
The first step is to compute \$TT^{-1}\$. Since the dimenions for each "inner" matrix is we can multiply

\$TT^{-1} = \begin{bmatrix} R & t \\ 0_{1X2} & 1 \end{bmatrix} \cdot \begin{bmatrix} R^T & -R^Tt \\ 0_{1X2} & 1 \end{bmatrix} = \begin{bmatrix} RR^T + (t \cdot 0_{1X2}) & (R \cdot(-R^Tt)+t) \\ (0_{1X2}\cdot R^T) + 0_{1X2} & (0_{1X2}\cdot(-R^Tt)+1) \end{bmatrix} = \begin{bmatrix} RR^T + (t \cdot 0_{1X2}) & (-RR^Tt)+t \\ (0_{1X2}\cdot R^T) + 0_{1X2} & (0_{1X2}\cdot-R^Tt+1) \end{bmatrix}\$

Computing the different "inner" matrix multiplications

\$RR^T = \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix} \cdot \begin{bmatrix}\cos{\theta} & \sin{\theta}\\ -\sin{\theta} & \cos{\theta}\end{bmatrix} =  \begin{bmatrix}{\cos^2{\theta}+\sin^2{\theta} } & {\cos{\theta}\sin{\theta}} -
{\cos{\theta}\sin{\theta}} \\ {\cos{\theta}\sin{\theta}} -{\cos{\theta}\sin{\theta}} & {\cos^2{\theta}+\sin^2{\theta} }\end{bmatrix} = \begin{bmatrix} 1 && 0\\ 0 && 1 \end{bmatrix} = I_2 \$

\$(t \cdot 0_{1X2}) = \begin{bmatrix} x \\ y \end{bmatrix} \begin{bmatrix} 0 & 0 \end{bmatrix} = \begin{bmatrix} 0 & 0 \\ 0 & 0 \end{bmatrix}\$

\$RR^T \cdot t = I_2\cdot t = \begin{bmatrix} 1 && 0\\ 0 && 1 \end{bmatrix} \cdot \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} x \\ y \end{bmatrix} \$

\$(0_{1X2} \cdot R^T) = \begin{bmatrix} 0 & 0 \end{bmatrix} \begin{bmatrix} cos(\theta) & sin(\theta) \\ -sin(\theta) & cos(\theta)  \end{bmatrix} = \begin{bmatrix} 0 & 0 \end{bmatrix}\$


\$(0_{1X2}\cdot-R^Tt) = \begin{bmatrix} 0 & 0 \end{bmatrix} \cdot -\begin{bmatrix} cos(\theta) & sin(\theta) \\ -sin(\theta) & cos(\theta) \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} 0 & 0 \end{bmatrix}  \cdot \begin{bmatrix} -xcos(\theta)-ysin(\theta) \\ xsin(\theta)-ycos(\theta) \end{bmatrix} = 0(-xcos(\theta)-ysin(\theta))+ 0(xsin(\theta)-ycos(\theta)) = 0  \$


This means we can express \$TT^{-1}\$ as

\$ TT^{-1} = \begin{bmatrix} I_2 & (-t+t) \\ 0_{1X2} & 1 \end{bmatrix} = \begin{bmatrix} I_2 & 0 \\ 0_{1X2} & 1 \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix} = I_3  \$



\$\\ \$
The next step is to compute $T^{-1}T$. Since the dimenions for each "inner" matrix is we can multiply

\$T^{-1}T = \begin{bmatrix} R^T & -R^Tt \\ 0_{1X2} & 1 \end{bmatrix} \cdot \begin{bmatrix} R & t \\ 0_{1X2} & 1 \end{bmatrix} = \begin{bmatrix} R^TR + (-R^Tt\cdot0_{1X2}) & (R^Tt -R^Tt) \\ (0_{1X2}\cdot R) + 0_{1X2} & (0_{1X2}\cdot t)+1) \end{bmatrix} \$

Computing the different "inner" matrix multiplications

\$R^TR = \begin{bmatrix}\cos{\theta} & \sin{\theta}\\ -\sin{\theta} & \cos{\theta}\end{bmatrix} \cdot \begin{bmatrix}\cos{\theta} & -\sin{\theta}\\ \sin{\theta} & \cos{\theta}\end{bmatrix} =  \begin{bmatrix}{\cos^2{\theta}+\sin^2{\theta} } & -{\cos{\theta}\sin{\theta}} +{\cos{\theta}\sin{\theta}} \\ -{\cos{\theta}\sin{\theta}} +{\cos{\theta}\sin{\theta}} & {\cos^2{\theta}+\sin^2{\theta} }\end{bmatrix} = \begin{bmatrix} 1 && 0\\ 0 && 1 \end{bmatrix} = I_2\$

\$(-R^Tt\cdot 0_{1X2}) = \begin{bmatrix} -xcos(\theta)-ysin(\theta) \\ xsin(\theta)-ycos(\theta) \end{bmatrix} \cdot \begin{bmatrix} 0 & 0 \end{bmatrix} = \begin{bmatrix} 0 & 0 \\ 0 & 0 \end{bmatrix}\$

\$(0_{1X2} \cdot R) = \begin{bmatrix} 0 & 0 \end{bmatrix} \begin{bmatrix} cos(\theta) & -sin(\theta) \\ sin(\theta) & cos(\theta)  \end{bmatrix} = \begin{bmatrix} 0 & 0 \end{bmatrix}\$

\$(0_{1X2} \cdot t)  = \begin{bmatrix} 0 & 0 \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} = 0\$


This means we can express \$TT^{-1}\$ as

\$ T^{-1}T = \begin{bmatrix} I_2 & 0 \\ 0_{1X2} & 1 \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix} = I_3  \$

These results show that \$TT^{-1}=T^{-1}T = I_3\$ and proves that \$T^{-1}\$ infact is the invertible matrix of \$T\$




### Exercise 6

a) Determine the associated homogeneous transformation matrices \$B_{T_{W}} B_{T_{T}}\$


In [26]:
# To find the homogeneous transformation matrices we use se2(x,y,theta) from Robotteknikk.py

# BtW, x = 3 y = 1 theta = -20 degrees
BtW = se2(3,1,np.radians(-20))
BtW

array([[ 0.93969262,  0.34202014,  3.        ],
       [-0.34202014,  0.93969262,  1.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [27]:
# BtT, x = 5 y = 4 theta = 10 degrees 
BtT = se2(5,4,np.radians(10))
BtT

array([[ 0.98480775, -0.17364818,  5.        ],
       [ 0.17364818,  0.98480775,  4.        ],
       [ 0.        ,  0.        ,  1.        ]])




b) Using \$B_{T_{W}} B_{T_{T}} \$ determine \$W_{T_{T}}\$

We can use the rule that \$\ominus ^B{T_{W}} = {^WT_{B}} \rightarrow {^WT_B} = (^BT_W)^{-1} \$ and with this we can determine \$W_{T_{T}} = W_{T_{B}} \oplus B_{T_{T}} \$

To compute this we use:

In [28]:
WtB = np.linalg.inv(BtW) # By finding the inverse matrix of BtW we get the homogeneous transformation matrix WtB
                         # which basically is the same vector path, but different direction
WtB

array([[ 0.93969262, -0.34202014, -2.47705772],
       [ 0.34202014,  0.93969262, -1.96575305],
       [ 0.        ,  0.        ,  1.        ]])




With this new homogeneous transformation matrix it is possible to determine \${^WT_{T}} = {^WT_{B}} \oplus {^BT_{T}} \$. 

To compute this we use:

In [29]:
WtT = WtB.dot(BtT)
WtT

array([[ 0.8660254 , -0.5       ,  0.85332481],
       [ 0.5       ,  0.8660254 ,  3.50311815],
       [ 0.        ,  0.        ,  1.        ]])



c) Determine the coordinates of the point 𝑝 in frame {𝑊} denoted by \$^Wp\$

From the frame of reference {W} the coordinates to \$^Wp = \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} 3 \\ 2 \end{bmatrix}\$ 



In [30]:
Wp = np.array([3,2])
Wp

array([3, 2])




d) Using \$^BT_W\$ and \${^Wp}\$, determine \$^Bp\$

To find \$^Bp\$ we can use the following equation

\$^Bp = {^BT_W}\cdot{^Wp} \$

The only problem is that the matrix dimensions are not correct for matrix multiplication. Therefore we have convert \$^Wp\$ to its homogenous form \$^W\tilde{p}\$, notice that we use \$\sim\$ denote this.

\$^Wp = \begin{bmatrix} 3 \\ 2 \end{bmatrix} \rightarrow {^W\tilde{p}} =  \begin{bmatrix} 3 \\ 2 \\ 1 \end{bmatrix}\$


In [31]:
Wp_h = e2h(Wp) # converts from 2D vector to 2D vector in homogeneous form by adding 1

Bp_h = BtW.dot(Wp_h) # Determines the 2D vector Bp in its homogeneous form
Bp = h2e(Bp_h) # converts to 2D vector, i.e removing the last row.
Bp

array([6.50311815, 1.85332481])